<a href="https://colab.research.google.com/github/uttamsingh0107/main/blob/master/GeneticAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pickle
import os
from multiprocessing import Pool
import sys
import numpy as np
import pandas as pd
from scipy.stats import skew,kurtosis
import keras
from keras.utils import np_utils
from keras.models import load_model,Model
from keras.models import Sequential  
from keras.layers import MaxPooling2D,Input,LSTM,Bidirectional
from keras.layers import Flatten,Dense, Dropout,BatchNormalization
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import LeaveOneOut as  lo
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from sklearn.preprocessing import OneHotEncoder
from keras.models import load_model as load

Using TensorFlow backend.


In [0]:
nmean,nmedian,nmax,nmin,nstd,nvar,nptp,nskew,nkurtosis=0,0,0,0,0,0,0,0,0
input_dataset=[]
input_labels=[]
for m in range(32):
    if (m<9):
        with open('/content/drive/My Drive/dataset/s0'+str(m+1)+'.dat','rb')as f:
            x=pickle.load(f,encoding='latin1')  
    else:
        with open('/content/drive/My Drive/dataset/s'+str(m+1)+'.dat','rb')as f:
            x=pickle.load(f,encoding='latin1')  

    for i in range(40):
       img=[]
       input_labels.append(x['labels'][i])
       for j in range(40):
           batch=x['data'][i][j]
           img.append(batch)
       input_dataset.append(img)
      
input_dataset=np.array(input_dataset) 
input_labels=np.array(input_labels)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-7649fa119545>", line 7, in <module>
    x=pickle.load(f,encoding='latin1')
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.py", l

KeyboardInterrupt: ignored

In [0]:
nmean,nmedian,nmax,nmin,nstd,nvar,nptp,nskew,nkurtosis=0,0,0,0,0,0,0,0,0
input_dataset=[]
input_labels=[]
for m in range(32):
    if (m<9):
        with open('/content/drive/My Drive/dataset/s0'+str(m+1)+'.dat','rb')as f:
            x=pickle.load(f,encoding='latin1')  
    else:
        with open('/content/drive/My Drive/dataset/s'+str(m+1)+'.dat','rb')as f:
            x=pickle.load(f,encoding='latin1')  

    for i in range(40):
       img=[]
       input_labels.append(x['labels'][i])
       for j in range(40):
           batch=[]
           for k in range(63):
               if(k!=62):
                   b=x['data'][i][j][128*k:128*(k+1)]
               else: 
                   b=x['data'][i][j][128*62:8064]
               batch.append(np.mean(b))
               nmean=nmean+np.mean(b)
               batch.append(np.median(b))
               nmedian=nmedian+np.median(b)
               batch.append(np.max(b))
               nmax=nmax+np.max(b)
               batch.append(np.min(b))
               nmin=nmin+np.min(b)
               batch.append(np.std(b))
               nstd=nstd+np.std(b)
               #batch.append(np.var(b))
               '''nvar=nvar+np.var(b)
               batch.append(np.ptp(b))
               nptp=nptp+np.ptp(b)
               batch.append(skew(b))
               nskew=nskew+skew(b)
               batch.append(kurtosis(b))
               nkurtosis=nkurtosis+kurtosis(b)'''
               if (k==62):
                   batch.append(nmean)
                   batch.append(nmedian)
                   batch.append(nmax)
                   batch.append(nmin)
                   batch.append(nstd)
                   '''batch.append(nvar)
                   batch.append(nptp)
                   batch.append(nskew)
                   batch.append(nkurtosis)'''
                   batch.append(i)
                   batch.append(m)
           img.append(batch)
           nmean,nmedian,nmax,nmin,nstd,nvar,nptp,nskew,nkurtosis=0,0,0,0,0,0,0,0,0
       input_dataset.append(img)
      
input_dataset=np.array(input_dataset) 
input_labels=np.array(input_labels)

In [0]:
print(input_dataset.shape)
print(input_labels.shape)

(1280, 40, 322)
(1280, 4)


In [0]:
import random
SEED = 2018
random.seed(SEED)
np.random.seed(SEED)


In [0]:
for i in range(1280):
    for j in range(4):
        if(input_labels[i][j]<=4):
            input_labels[i][j]=0
        if(input_labels[i][j]>4 and input_labels[i][j]<=6):
            input_labels[i][j]=1
        else:
            input_labels[i][j]=2
input_labels1=input_labels
input_labels=input_labels[:,0:1]

In [0]:
print(input_labels.shape)

(1280, 1)


In [0]:
def est(inp,pred):
    for i in range(24):
      '''X_train,X_test,y_train,y_test=train_test_split(input_dataset,input_labels,test_size=0.1,random_state=6)
      X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.03472,random_state=5)'''
      X_train=[]
      y_train=[]
      X_val=[]
      y_val=[]
      X_test=inp[i*40:(i+1)*40,:,:]
      y_test=pred[i*40:(i+1)*40,:]
      print(X_test.shape)
      for j in range(32):
          if(j!=i):
            X_train.append(inp[j*40:(j+1)*40,:,:])
            y_train.append(pred[j*40:(j+1)*40,:])
      X_val=X_test
      y_val=y_test
      X_train=np.array(X_train) 
      y_train=np.array(y_train) 
      X_train=np.reshape(X_train,(X_train.shape[0]*X_train.shape[1],inp.shape[1],322))
      y_train=np.reshape(y_train,(y_train.shape[0]*y_train.shape[1],1))
      model=Sequential()
      model.add(Bidirectional(LSTM(128,return_sequences=True),input_shape=(inp.shape[1],322)))
      model.add(Dropout(0.3))
      model.add(BatchNormalization())
      #model.add(LSTM(64,return_sequences=True))
      model.add(Flatten())
      model.add(Dense(128,activation='relu'))
      model.add(Dropout(0.3))
      model.add(BatchNormalization())
      model.add(Dense(3,activation='softmax'))
      model.summary()
    # sgd = SGD(lr = 0.00001, momentum = 0.9, nesterov = True)
      mcp_save = ModelCheckpoint('model-{epoch:03d}-{acc:03f}-{val_acc:03f}.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')
      model.compile( loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'] )
      
      #Fitting the data to the model
      history=model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1, callbacks=[mcp_save], validation_data=(X_val, y_val))
    return history.history['val_loss']

In [0]:
class geneticSelector():
  def __init__(self,estimator,n_gen,size,n_best,n_rand,n_children,mutation_rate):
    self.estimator=estimator
    self.n_children=n_children
    self.n_gen=n_gen
    self.size=size
    self.n_best=n_best
    self.n_rand=n_rand
    self.mutation_rate=mutation_rate
    if int((n_best+n_rand)/2)*self.n_children != self.size:
            raise ValueError("The population size is not stable.") 
  
  
  def initialize(self):
     population = []
     for i in range(self.size):
            chromosome = np.ones(self.n_features, dtype=np.bool)
            mask = np.random.rand(len(chromosome)) < 0.3
            chromosome[mask] = False
            population.append(chromosome)
     return population
  def fitness(self,population):
    X,Y=input_dataset,input_labels
    scores=[]
    print(population)
    for chromosome in population:
      score=(self.estimator( X[:,chromosome,:], Y))
      print(score)
      scores.append(score)
    scores,population =np.array(scores),np.array(population)
    inds=np.argsort(scores)
    return list(scores[inds]), list(population[inds,:])
  def select(self,population_sorted):
    population_next=[]
    for i in range(self.n_best):
      population_next.append(population_sorted[i])
    for i in range(self.n_rand):
      population_next.append(random.choice(population_sorted))
    random.shuffle(population_next)
    return population_next
  def crossover(self,population):
    population_next=[]
    for i in range(int(len(population)/2)):
      for j in range(self.n_children):
        chromosome1,chromosome2=population[i],population[len(population)-1-i]
        child=chromosome1
        mask=np.random.rand(len(child))>0.5
        child[mask]=chrosome2[mask]
        population_next.append(child)
    return population_next
  def mutate(self,population):
   population_next=[]
   for i in range(len(population)):
     chrosome=population[i]
     if random.random() < self.mutation_rate:
       mask=np.random.rand(len(chromosome)) < 0.05
       chromosome[mask]=False
     population_next.append(chromosome)
   return population_next
  def generate(self,population):
   scores_sorted,population_sorted=self.fitness(population)
   population=self.select(population_sorted)
   population=self.crossover(population)
   population=self.mutate(population)
   self.chromosomes_best.append(population_sorted[0])
   self.score_best.append(scores_sorted[0])
   self.scores_avg.append(np.mean(scores_sorted))
   return population
  def fit(self,X,Y):
    self.chromosomes_best = []
    self.scores_best, self.scores_avg  = [], []
    self.dataset = X, Y
    self.n_features = X.shape[1]
    population = self.initialize()
    for i in range(self.n_gen):
        population = self.generate(population)  
    return self 
  @property
  def support_(self):
    return self.chromosomes_best[-1]
sel = geneticSelector(estimator=est, 
                      n_gen=3, size= 6, n_best=3, n_rand=3, 
                      n_children=2, mutation_rate=0.05)
sel.fit(input_dataset, input_labels)
#sel.plot_scores()

        




Streaming output truncated to the last 5000 lines.
Epoch 00037: val_loss did not improve from 0.64287
Epoch 38/50
1240/1240 [==============================] - 4s 3ms/step - loss: 0.5499 - acc: 0.7339 - val_loss: 0.5779 - val_acc: 0.7250

Epoch 00038: val_loss improved from 0.64287 to 0.57788, saving model to model-038-0.733871-0.725000.h5
Epoch 39/50
1240/1240 [==============================] - 4s 3ms/step - loss: 0.5526 - acc: 0.7323 - val_loss: 0.6443 - val_acc: 0.6250

Epoch 00039: val_loss did not improve from 0.57788
Epoch 40/50
1240/1240 [==============================] - 4s 3ms/step - loss: 0.5334 - acc: 0.7403 - val_loss: 0.6430 - val_acc: 0.6250

Epoch 00040: val_loss did not improve from 0.57788
Epoch 41/50
1240/1240 [==============================] - 4s 3ms/step - loss: 0.5295 - acc: 0.7460 - val_loss: 0.7581 - val_acc: 0.6750

Epoch 00041: val_loss did not improve from 0.57788
Epoch 42/50
1240/1240 [==============================] - 4s 3ms/step - loss: 0.5520 - acc: 0.7315 